探索中の予測モデルのlossをデータセットとしてまとめる

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np

In [ ]:
import re

def get_model_select(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
    pattern_task_info=r"(\d+\.?\d*) ([A-Za-z_]+) ([a-z_]+)"
    pattern_selected_model=r"選択したモデル：(.+), val_loss：(\d+\.?\d*)"

    task_info_list=[]
    selected_model_list=[]
    for line in lines:
        match_ti=re.match(pattern_task_info,line)
        if match_ti:
            task_info_list.append((float(match_ti.group(1)),match_ti.group(2),match_ti.group(3)))
        match_ml=re.match(pattern_selected_model,line)
        if match_ml:
            selected_model_list.append((match_ml.group(1),float(match_ml.group(2))))
    
    if len(task_info_list)!=len(selected_model_list):
        print("ERROR")
        return None

    result_model_selection={}
    for i in range(len(task_info_list)):
        if not (task_info_list[i][1] in result_model_selection.keys()):
            result_model_selection[task_info_list[i][1]]=[]

        # (実行モード,選択されたモデル,モデルのval_loss)
        result_model_selection[task_info_list[i][1]].append((task_info_list[i][2],selected_model_list[i][0],selected_model_list[i][1]))
    return result_model_selection


### val_lossの比較と選択したモデル

In [ ]:
import os
def plot_loss_and_model(obj_name,seed,random_search,path_list,label_list):
    result_model_selection=[]
    for path in path_list:
        result_model_selection.append(get_model_select(os.path.join(path,f"stdout{seed}")))

    result_lists=[]
    for rms in result_model_selection:
        result_lists.append(rms[f"BO_system_{obj_name}"])

    for i, result_list in enumerate(result_lists):
        plt.plot(np.arange(random_search,random_search+len(result_list)),[result_list[i][2] for i in range(len(result_list))], c=f"C{i}", label=label_list[i])

    # 転移学習モデルの表示
    color_dict={"no_transfer":"C3","transfer_CT":"C4","transfer_MM":"C5","transfer_SP":"C6"}
    model_count_list=[]
    for result_list in result_lists:
        model_count=dict()
        for model in color_dict.keys():
            for i in range(len(result_list)):
                if result_list[i][0]=="transfer" and model==result_list[i][1]:
                    if result_list[i][1] in model_count.keys():
                        plt.plot(i+random_search,result_list[i][2],"o",c=color_dict[result_list[i][1]])
                        model_count[result_list[i][1]]+=1
                    else:
                        plt.plot(i+random_search,result_list[i][2],"o",c=color_dict[result_list[i][1]], label=result_list[i][1])
                        model_count[result_list[i][1]]=1
        model_count_list.append(model_count)

        # 知見共有を行った点
        for j in range(len(result_list)):
            if result_list[j][0]=="transfer":
                plt.axvline(x=random_search+j, color='gray', linestyle='--', alpha=0.2)
    # 探索開始の表示
    plt.axvline(x=random_search, color='gray', linestyle='-', alpha=0.5)


    plt.xlabel("探索回数")
    plt.ylabel("選択されたモデルのval_loss")
    plt.title(f"選択されたモデルのval_lossの比較{seed} ({obj_name})")

    plt.xlim(-5,len(result_lists[0])+random_search)

    

    plt.legend()
    plt.show()

    return model_count_list

In [ ]:
import pandas as pd

In [ ]:
path="result_network9_6"
filename="stdout0"

result=get_model_select(os.path.join(path,filename))
df=pd.DataFrame(result["BO_system_MM"],columns=["mode","select_model","val_loss"])
df

In [ ]:
network_name="result_network9_6"
pattern_size=10

output_path=os.path.join("preparation_results",network_name)
if not os.path.exists(output_path):
    os.makedirs(output_path)


for seed in range(pattern_size):
    filename="stdout"+str(seed)
    result=get_model_select(os.path.join(path,filename))
    for name,data in result.items():
        if not os.path.exists(os.path.join(output_path,name)):
            os.makedirs(os.path.join(output_path,name))

        df=pd.DataFrame(data,columns=["mode","select_model","val_loss"])
        df.to_csv(os.path.join(os.path.join(output_path,name),f"val_loss{seed}.csv"),index=False)



In [ ]:
# obj_name="SP"
# result_num=10
# random_search=0
# # path_list=["result_network8","result_network7","result_network12"]
# # label_list=["知見共有なし","知見共有あり(周期5)","知見共有あり(周期1)"]
# path_list=["result_network10","result_network9","result_network11"]
# label_list=["知見共有なし","知見共有あり(周期1)","知見共有あり(周期5)"]

# sum_model_count={}
# for i in range(result_num):
#     model_count=plot_loss_and_model(obj_name,i,random_search,path_list,label_list)
#     print(model_count)
# #     for model in model_count.keys():
# #         if model in sum_model_count:
# #             sum_model_count[model]+=model_count[model]
# #         else:
# #             sum_model_count[model]=model_count[model]
# # print("選択されたモデルの合計数\n",sum_model_count)

unobservedの予測性能データセットを作成、プロットする

In [ ]:
network_name="result_network9_6"
pattern_size=10

output_path=os.path.join("preparation_results",network_name)
if not os.path.exists(output_path):
    os.makedirs(output_path)

system_name_list=["BO_system_CT","BO_system_MM","BO_system_SP"]

for system_name in system_name_list:
    for seed in range(pattern_size):
        df = pd.read_csv(os.path.join(os.path.join(output_path,system_name),f"val_loss{seed}.csv"))
        df_unobserved = pd.read_csv(os.path.join(network_name,system_name,f"unobserved_prediction{seed}.csv"))
        concat = pd.concat([df[["mode","select_model"]],df_unobserved[["MAE"]]],axis=1)
        concat.to_csv(os.path.join(os.path.join(output_path,system_name),f"predict_loss_unobserved{seed}.csv"),index=False)


In [ ]:
import os
def plot_loss_and_model(obj_name,seed,random_search,path_list,label_list):
    result_model_selection=[]
    for path in path_list:
        result_model_selection.append(pd.read_csv(os.path.join("preparation_results",path,f"BO_system_{obj_name}",f"predict_loss_unobserved{seed}.csv")))
        # result_model_selection.append(get_model_select(os.path.join(path,f"stdout{seed}")))

    result_lists=[]
    for rms in result_model_selection:
        result_lists.append(list(rms.itertuples(index=False,name=None)))

    for i, result_list in enumerate(result_lists):
        plt.plot(np.arange(random_search,random_search+len(result_list)),[result_list[i][2] for i in range(len(result_list))], c=f"C{i}", label=label_list[i])

    # 転移学習モデルの表示
    color_dict={"no_transfer":"C3","transfer_CT":"C4","transfer_MM":"C5","transfer_SP":"C6"}
    model_count_list=[]
    for result_list in result_lists:
        model_count=dict()
        for model in color_dict.keys():
            for i in range(len(result_list)):
                if result_list[i][0]=="transfer" and model==result_list[i][1]:
                    if result_list[i][1] in model_count.keys():
                        plt.plot(i+random_search,result_list[i][2],"o",c=color_dict[result_list[i][1]])
                        model_count[result_list[i][1]]+=1
                    else:
                        plt.plot(i+random_search,result_list[i][2],"o",c=color_dict[result_list[i][1]], label=result_list[i][1])
                        model_count[result_list[i][1]]=1
        model_count_list.append(model_count)

        # 知見共有を行った点
        for j in range(len(result_list)):
            if result_list[j][0]=="transfer":
                plt.axvline(x=random_search+j, color='gray', linestyle='--', alpha=0.2)
    # 探索開始の表示
    plt.axvline(x=random_search, color='gray', linestyle='-', alpha=0.5)


    plt.xlabel("探索回数")
    plt.ylabel("選択されたモデルのunobserved予測誤差")
    plt.title(f"選択されたモデルのunobserved予測誤差の比較{seed} ({obj_name})")

    plt.xlim(-5,len(result_lists[0])+random_search)

    

    plt.legend()
    plt.show()

    return model_count_list

In [ ]:
obj_name="MM"
result_num=10
random_search=10

path_list=["result_network10_6","result_network9_6"]
label_list=["知見共有なし","知見共有あり(周期1)"]

sum_model_count={}
for i in range(result_num):
    model_count=plot_loss_and_model(obj_name,i,random_search,path_list,label_list)
    print(model_count)
#     for model in model_count.keys():
#         if model in sum_model_count:
#             sum_model_count[model]+=model_count[model]
#         else:
#             sum_model_count[model]=model_count[model]
# print("選択されたモデルの合計数\n",sum_model_count)